# Sheath Expansion - Fixed Shadow Simulations Results - Analysis Round 1 (Reprise)

Checks of shexp_shad simluations checking that the shadow problem has been remedied

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm
import flopter.spice.xrfuncs as xrf

In [2]:
importlib.reload(fts)

<module 'flopter.core.fitters' from '/home/jleland/coding/projects/flopter/flopter/core/fitters.py'>

## Paths and Matfile IO
This section deals with file io and selecting the right .mat files. This needs to be run for the latter sections to work.

In [3]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
os.chdir(spice_dir)

In [4]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [5]:
skippable_runs = set([
    'marconi/spice2/sheath_exp_hg/flat_flush_gapfill/alpha_yz_-6.0', # accidental duplicate
    'marconi/spice2/sheath_exp_hg/angled_recessed_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_hg/angled_recessed_as/alpha_yz_-3.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-3.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-4.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-5.0',
    'marconi/spice2/sheath_exp_hg_fflwp/angled_recessed_as/alpha_yz_-6.0',
    'marconi/spice2/sheath_exp_hg_fflwp/flat_flush_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_fflwp/angled_recessed_as/alpha_yz_-2.0',
    'marconi/spice2/sheath_exp_fwp/flat_flush_wp-2_as_1_/alpha_yz_-2.0',
    'marconi/spice2/bergmann_bm/flat_flush_lowas/alpha_yz_-1.5',
]) 

single_sims = set()
skippable_scans = {
    'marconi/spice2/sheath_exp/angled',
    'marconi/spice2/sheath_exp_fwp/flat_10',
    'marconi/spice2/sheath_exp_fwp/flat_10_highV',
    'marconi/spice2/sheath_exp_fwp/flat_8-12_highV',
} 
single_sims = set()

In [6]:
non_standard_variables = {'t', 'WallPot', 'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc', 'rho', 'rho01', 'rho02'}
desired_variables = (td.DEFAULT_REDUCED_DATASET | non_standard_variables) #- {td.OBJECTSCURRENTFLUXE, td.OBJECTSCURRENTFLUXI}

In [7]:
all_angles = {'-2.0', '-3.0', '-4.0', '-5.0', '-6.0', '-7.0', '-8.0', '-9.0',
              '10.0', '11.0', '12.0', '14.0', '15.0', '16.0', '18.0', '20.0',
              '25.0', '30.0'}
# allowed_angles = ['-2.0', '-3.0', '-5.0', '-8.0', '10.0', '30.0']
allowed_angles = all_angles
disallowed_angles = all_angles - allowed_angles

In [32]:

scans_searchstr = [
#     '*/*/piecewise/*',
#     '*/*/shexp_shad_test/*',
#     '*/*/shexp_shad_min/*',
    '*/*/shexp_shad_fflwp/*',
#     '*/*/shexp_shad_fwp0/*',
]
disallowed_angles = []

scans, all_run_dirs = xrf.get_run_dirs(scans_searchstr, skippable_runs=skippable_runs, #angles_search_str='/ta_*', 
                                       disallowed_angles=disallowed_angles)


[0]: marconi/spice2/shexp_shad_fflwp/angled_recessed_as
	[0,0]: angled_recessed_as/alpha_yz_-11.0
	[0,1]: angled_recessed_as/alpha_yz_-12.0
	[0,2]: angled_recessed_as/alpha_yz_-14.0
	[0,3]: angled_recessed_as/alpha_yz_-16.0
	[0,4]: angled_recessed_as/alpha_yz_-18.0
	[0,5]: angled_recessed_as/alpha_yz_-2.0
	[0,6]: angled_recessed_as/alpha_yz_-20.0
	[0,7]: angled_recessed_as/alpha_yz_-25.0
	[0,8]: angled_recessed_as/alpha_yz_-3.0
	[0,9]: angled_recessed_as/alpha_yz_-30.0
	[0,10]: angled_recessed_as/alpha_yz_-5.0
	[0,11]: angled_recessed_as/alpha_yz_-7.0
	[0,12]: angled_recessed_as/alpha_yz_-8.0
	[0,13]: angled_recessed_as/alpha_yz_-9.0
[1]: marconi/spice2/shexp_shad_fflwp/flat_flush_as
	[1,0]: flat_flush_as/alpha_yz_-11.0
	[1,1]: flat_flush_as/alpha_yz_-12.0
	[1,2]: flat_flush_as/alpha_yz_-14.0
	[1,3]: flat_flush_as/alpha_yz_-16.0
	[1,4]: flat_flush_as/alpha_yz_-18.0
	[1,5]: flat_flush_as/alpha_yz_-2.0
	[1,6]: flat_flush_as/alpha_yz_-20.0
	[1,7]: flat_flush_as/alpha_yz_-25.0
	[1,8]: flat

---
## Function For Per-run Analysis

In [33]:
spl_path = spice_dir / all_run_dirs[scans[0]][9]
print(spl_path)
spl_backup = list(spl_path.glob('backup_20*'))[-1]
print(spl_backup.name)

/home/jleland/data/external_big/spice/marconi/spice2/shexp_shad_fflwp/angled_recessed_as/alpha_yz_-30.0
backup_20210321-0936


In [34]:
splopter =  spl.Splopter(spl_backup, reduce=None, ignore_tzero_fl=True, version=2.14,
                         store_dataframe_fl=True, check_voltage_error_fl=False, ignore_a_file=True)
print(splopter.tdata.dt)

parser = inp.InputParser(input_filename=spl_backup / 'input.inp')
denormaliser = nrm.Denormaliser(dt=splopter.tdata.dt, input_parser=parser) #, dimensions=3)
splopter.parse_input()

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/shexp_shad_fflwp/angled_recessed_as/alpha_yz_-30.0/backup_20210321-0936) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "Esct" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "dPHIqn" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "pchi" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages

No a-file given, continuing without
[[0.00048507]]
dx = 1.0, dy = 1.0, dz = 1.0


In [35]:
# Test
iv_df, iv_data = xrf.homogenise_run(spl_backup)
# iv_df = iv_df.drop(iv_df.index[-1])
iv_df

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/shexp_shad_fflwp/angled_recessed_as/alpha_yz_-30.0/backup_20210321-0936) doesn't seem to be valid.
Continuing anyway.
No a-file given, continuing without


,voltage,voltage_wall,current,current_e,current_i,d_current,d_current_e,d_current_i,time
0,-14.975,-2.800000,-111.307692,0.000000,111.307692,4.112808,0.000000,4.112808,0.000000
1,-14.950,-2.361922,-116.384615,0.000000,116.384615,3.263116,0.000000,3.263116,0.006306
2,-14.925,-2.361922,-116.076923,0.000000,116.076923,3.024391,0.000000,3.024391,0.012612
3,-14.875,-2.025381,-117.846154,0.000000,117.846154,2.640528,0.000000,2.640528,0.018918
4,-14.850,-2.025381,-111.000000,0.000000,111.000000,3.557225,0.000000,3.557225,0.025224
...,...,...,...,...,...,...,...,...,...
793,4.875,-1.847694,408.881048,-411.188740,2.307692,5.163111,5.259466,0.458322,5.000562
794,4.900,-1.845056,423.423359,-425.346436,1.923077,4.570375,4.687799,0.430586,5.006868
795,4.925,-1.840835,451.969542,-453.738773,1.769231,4.572419,4.660885,0.378151,5.013174
796,4.950,-1.840835,483.362882,-485.824421,2.461538,8.120031,8.048835,0.432870,5.019480


In [36]:
fig, ax = plt.subplots()
iv_df.set_index('voltage')['current'].plot()
# ax.set_ylim([-30,0])

In [23]:
fig, ax = plt.subplots()
iv_df['voltage'].plot()
iv_df['voltage_wall'].plot()

vw_mean = iv_df['voltage_wall'].mean()
vw_std = iv_df['voltage_wall'].std()
ax.axhline(y=vw_mean, **c.AX_LINE_DEFAULTS)
ax.axhspan(vw_mean - vw_std, vw_mean + vw_std, color='silver', alpha=0.7)

In [24]:
fig, ax = plt.subplots()

ax.plot(splopter.tdata.t_dict['ProbePot'])
ax.plot(splopter.tdata.t_dict['WallPot'])

for i in range(4):
    line_pos = i * len(splopter.tdata.t_dict['ProbePot']) / 4
    ax.axvline(x=line_pos, **c.AX_LINE_DEFAULTS)

In [25]:
t, V, I_e, I_i = splopter.get_tdata_raw_iv()
V_wall = splopter.get_wall_potential()

fig, ax = plt.subplots(2, sharex=False, figsize=[10,6])
I_tot = (I_e + I_i)[::100]
ax[0].plot(I_tot)
ax[1].plot(V)
ax[1].plot(V_wall)

for axis in ax:
    for i in range(4):
        line_pos = i * len(I_tot) / 4
        axis.axvline(x=line_pos, **c.AX_LINE_DEFAULTS)



In [26]:
fig, ax = plt.subplots(2)
ax[0].pcolormesh(splopter.tdata.t_dict['rho'][:-5,:])
ax[1].pcolormesh(splopter.tdata.t_dict['Pot'][:-5,:])

In [18]:
o_file = sio.loadmat(list(spl_backup.glob('alpha*'))[0])
print(list(o_file.keys()))

['__header__', '__version__', '__globals__', 'version', 'nspec', 'Nz', 'Ny', 'Np', 'nav', 'dt', 'dz', 'angbin', 'z', 'y', 'Epar', 'Eperp', 'Potav', 'dens01', 'q01', 'T01', 'vzav01', 'vyav01', 'vxav01', 'vz2av01', 'vy2av01', 'vx2av01', 'vz3av01', 'vy3av01', 'vx3av01', 'temperature01', 'edgeflux01', 'edgeenergyflux01', 'edgevelocityx01', 'edgevelocityy01', 'edgevelocityz01', 'dens02', 'q02', 'T02', 'vzav02', 'vyav02', 'vxav02', 'vz2av02', 'vy2av02', 'vx2av02', 'vz3av02', 'vy3av02', 'vx3av02', 'temperature02', 'edgeflux02', 'edgeenergyflux02', 'edgevelocityx02', 'edgevelocityy02', 'edgevelocityz02', 'N0']


In [19]:
fig, ax = plt.subplots(2)
ax[0].pcolormesh(splopter.tdata.t_dict['rho'][:-5,:])
ax[1].pcolormesh(o_file['edgeflux01'])


In [20]:
fig, ax = plt.subplots(2, 2, sharex=True, figsize=[12,12])
ax[0, 0].pcolormesh(o_file['edgeflux01'])
ax[0, 1].pcolormesh(o_file['edgeflux02'])

ax[1, 0].plot(o_file['edgeflux01'].sum(axis=0)) # + o_file['edgeflux02'].sum(axis=0))
ax[1, 1].plot(o_file['edgeflux02'].sum(axis=0))

In [21]:
fig, ax = plt.subplots(2, 2)
ax[0,0].pcolormesh(splopter.tdata.t_dict['edges'], cmap='plasma')
ax[1,0].pcolormesh(splopter.tdata.t_dict['objectsenum'])
print(splopter.tdata.t_dict['objectspowerfluxi'][0].shape)

ax[0,1].plot(splopter.tdata.t_dict['objectscurrenti'].sum(axis=0))
ax[1,1].plot(splopter.tdata.t_dict['objectscurrente'].sum(axis=0))



(1065356,)


In [22]:
fig, ax = plt.subplots()
mapper = ax.pcolormesh(splopter.tdata.pot)
fig.colorbar(mapper)


In [23]:
fig, ax = plt.subplots()
mapper = ax.pcolormesh(splopter.tdata.rho)
fig.colorbar(mapper)


In [24]:
fig, ax = plt.subplots(2, figsize=[10,12])

print(splopter.tdata.pot.shape)
ax[0].plot(splopter.tdata.rho[93:(93+36),0:1].mean(axis=1))
ax[1].plot(splopter.tdata.pot[:,0:1].mean(axis=1))
fig.tight_layout()

(233, 1373)


In [29]:
fig, ax = plt.subplots()
xrf.norm_ion_fit(iv_data, v_redund=0).plot(ax=ax)
xrf.norm_ion_fit(iv_data, v_redund=1, voltage_cap=-14).plot(ax=ax)

In [31]:
fig, ax = plt.subplots(1, 2)

f_data_whole = xrf.norm_ion_fit(iv_data, v_redund=0)
f_data_fit = xrf.norm_ion_fit(iv_data, v_redund=1, voltage_cap=-14)
ax[0].plot(f_data_whole.raw_x, f_data_whole.raw_y)


In [27]:
fig, ax = plt.subplots(2, sharex=True)

a_vals = []
d_a_vals = []
v_caps = np.linspace(-15.0, -4.0, 101)
v_redunds = np.linspace(0, 10, 101)

for v_cap in v_caps:
    f_data = xrf.norm_ion_fit(iv_data, sigma=iv_df['d_current_i'].values, v_redund=0, voltage_cap=v_cap)
    a_vals.append(f_data.get_sheath_exp())
    d_a_vals.append(f_data.get_sheath_exp_err())
ax[0].errorbar(v_caps, a_vals, yerr=d_a_vals)
ax[0].set_ylim(0.0, 0.2)


a_vals = []
d_a_vals = []
for v_redund in v_redunds:
    f_data = xrf.norm_ion_fit(iv_data, sigma=iv_df['d_current_i'].values, v_redund=v_redund, voltage_cap=-15.0)
    a_vals.append(f_data.get_sheath_exp())
    d_a_vals.append(f_data.get_sheath_exp_err())
ax[1].errorbar(-v_redunds, a_vals, yerr=d_a_vals)
# ax[1].set_ylim(0.0, 0.2)    

CAUGHT: ion fit runtime error


<ErrorbarContainer object of 3 artists>

# Testing the fit functions

In [47]:
importlib.reload(xrf)

<module 'flopter.spice.xrfuncs' from '/home/jleland/coding/projects/flopter/flopter/spice/xrfuncs.py'>

In [56]:
probe_ar = xrf.FLUSH_SQUARE_PROBE
probe_ar.theta_p = np.radians(10.0)

In [59]:
all_initial_params = xrf.get_initial_params(30.0)#, probe=probe_ar)
print(all_initial_params)
1 / denormaliser(1, 'current')

{'temperature': 1.0, 'floating_potential': -2.5, 'isat': 29.21082083390248, 'sheath_exp_param': 0.01}


3.50416594252353

In [49]:
fig, ax = plt.subplots()
xrf.norm_ion_fit(iv_data, v_redund=0.5).plot(ax=ax)


In [52]:
fig, ax = plt.subplots()
all_initial_params['isat'] = 120
xrf.straight_iv_fit(iv_data, cutoff=None, all_initial_values=all_initial_params).plot(ax=ax)